# Parsing the Causal Biological Network Database

This notebook outlines the process of parsing the JSON Graph File format used in the Causal Biological Network (CBN) Database. 

In [1]:
import json
import requests
import time

from IPython.display import IFrame, HTML
import networkx as nx
import pybel
from pybel.constants import EVIDENCE

In [2]:
pybel.__version__

'0.3.4'

In [3]:
time.asctime()

'Sun Jan 22 20:41:50 2017'

In [4]:
species_map = {
    'human': '9606',
    'rat': '10116',
    'mouse': '10090'
}

## Data Acquisition

In [5]:
res = requests.get("http://causalbionet.com/Networks/GetJSONGraphFile?networkId=hox_2.0_hs").json()

## Traversal and Parsing

In [6]:
graph = pybel.BELGraph()
parser = pybel.parser.BelParser(graph)

In [7]:
for edge in res['graph']['edges']:    
    for evidence in edge['metadata']['evidences']:
        if 'citation' not in evidence or not evidence['citation']:
            continue
        
        parser.control_parser.clear()
        parser.control_parser.citation = {
            'name': evidence['citation']['name'],
            'type': evidence['citation']['type'],
            'reference': evidence['citation']['id'],
        }
            
        d = {
            EVIDENCE: evidence['summary_text']
        }
        
        if 'biological_context' in evidence:
            annotations = evidence['biological_context']
        
            if annotations['tissue']:
                d['Tissue'] = annotations['tissue']

            if annotations['disease']:
                d['Disease'] = annotations['disease']

            if annotations['species_common_name']:
                d['Species'] = species_map[annotations['species_common_name'].lower()]
        
        parser.control_parser.annotations.update(d)
        bel = '{source} {relation} {target}'.format_map(edge)
        try:
            parser.parseString(bel)
        except:
            print(bel, d)

In [8]:
graph.number_of_nodes(), graph.number_of_edges()

(27, 44)